# 11. Neural Networks with MNIST

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
from torch.utils.data import DataLoader

import torchvision.datasets as dsets
import torchvision.transforms as transforms

import numpy as np
import os

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline

## 11.1 Prepare MNIST Data

In [3]:
# https://pytorch.org/docs/stable/torchvision/transforms.html

transform = transforms.Compose([
#           Data Augmentation
#           transforms.RandomRotation(15)
#           transforms.CenterCrop(28),
#           transforms.Lambda(lambda x: x.rotate(15)),
    
#           Data Nomalization
            transforms.ToTensor(),
            transforms.Normalize(mean=(0.5,), std=(0.5,))
# Normalize a tensor image with mean and standard deviation.
# Given mean: (M1,...,Mn) and std: (S1,..,Sn) for n channels,
# this transform will normalize each channel of the input torch.
# *Tensor i.e. input[channel] = (input[channel] - mean[channel]) / std[channel]
            ])


train_data = dsets.MNIST(root='data/',
                         train=True,
                         transform=transform,
                         download=True)

test_data = dsets.MNIST(root='data/',
                        train=False,
                        transform=transform,
                        download=True)

In [4]:
train_data.data.size(), test_data.data.size()

(torch.Size([60000, 28, 28]), torch.Size([10000, 28, 28]))

## 11.2 Make Batch Loader

In [5]:
batch_size = 100

train_loader  = DataLoader(dataset=train_data,
                           batch_size=batch_size,
                           shuffle=True,
                           num_workers=1)

test_loader  = DataLoader(dataset=test_data,
                           batch_size=batch_size,
                           shuffle=True,
                           num_workers=1)

## 11.3 Define Model

In [6]:
model = torch.nn.Sequential(
    torch.nn.Linear(784, 512),
    torch.nn.ReLU(),
    #Dropout
    nn.Dropout(0.5),
    
    torch.nn.Linear(512, 256),
    torch.nn.ReLU(),
    
    torch.nn.Linear(256, 64),
    #Batch Nomalization
    nn.BatchNorm1d(64),
    torch.nn.ReLU(),
    
    torch.nn.Linear(64, 10)
)

In [7]:
# Weight Initialization

for m in model.modules() :
    if isinstance(m, nn.Linear):
        # init.xavier_normal(m.weight.data)
        init.kaiming_normal_(m.weight.data)
        m.bias.data.fill_(0)                

In [8]:
loss = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Momentum & Weight Regularization(L2)
# optim.SGD(model.parameters(), lr=1e-2, momentum=0.9, weight_decay=1e-5)

## 11.4 Train Model

In [9]:
num_epochs = 10

In [10]:
def test_model() :
    
    model.eval()
    
    correct = 0
    total = 0

    for images, labels in test_loader :

        outputs = model(images.view(-1, 28 * 28))
        _, predicted = torch.max(outputs.data, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum()

    print('Accuracy of test images: %f %%' % (100 * float(correct) / total))

In [11]:
save_path = '/model/'

if not os.path.exists(save_path):
    os.makedirs(save_path)

In [12]:
for epoch in range(num_epochs):
    
    total_batch = len(train_data) // batch_size
    
    for i, (batch_images, batch_labels) in enumerate(train_loader):
        
        X = batch_images.view(-1, 28 * 28)
        Y = batch_labels
        
        pre = model(X)
        cost = loss(pre, Y)
        
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()
        
        if (i+1) % 300 == 0:
            print('Epoch [%d/%d], lter [%d/%d], Loss: %.4f'
                 %(epoch+1, num_epochs, i+1, total_batch, cost.item()))
            
            # Test Model
            test_model()
            
            # Save Model
            model_path = save_path + 'model_' + str(epoch+1) + '_' + str(i+1) + '.pth'
            print(model_path)
            torch.save(model.state_dict(), model_path)
            
            model.train()
    
print("Learning Finished!")

Epoch [1/10], lter [300/600], Loss: 0.3871
Accuracy of test images: 92.450000 %
/model/model_1_300.pth
Epoch [1/10], lter [600/600], Loss: 0.2112
Accuracy of test images: 94.850000 %
/model/model_1_600.pth
Epoch [2/10], lter [300/600], Loss: 0.2077
Accuracy of test images: 95.600000 %
/model/model_2_300.pth
Epoch [2/10], lter [600/600], Loss: 0.4839
Accuracy of test images: 95.970000 %
/model/model_2_600.pth
Epoch [3/10], lter [300/600], Loss: 0.1577
Accuracy of test images: 95.720000 %
/model/model_3_300.pth
Epoch [3/10], lter [600/600], Loss: 0.1697
Accuracy of test images: 96.400000 %
/model/model_3_600.pth
Epoch [4/10], lter [300/600], Loss: 0.2228
Accuracy of test images: 96.100000 %
/model/model_4_300.pth
Epoch [4/10], lter [600/600], Loss: 0.2678
Accuracy of test images: 96.260000 %
/model/model_4_600.pth
Epoch [5/10], lter [300/600], Loss: 0.2501
Accuracy of test images: 96.710000 %
/model/model_5_300.pth
Epoch [5/10], lter [600/600], Loss: 0.0652
Accuracy of test images: 96.82

## 11.5 Test Model

In [17]:
model = torch.nn.Sequential(
    torch.nn.Linear(784, 512),
    torch.nn.ReLU(),
    #Dropout
    nn.Dropout(0.5),
    
    torch.nn.Linear(512, 256),
    torch.nn.ReLU(),
    
    torch.nn.Linear(256, 64),
    #Batch Nomalization
    nn.BatchNorm1d(64),
    torch.nn.ReLU(),
    
    torch.nn.Linear(64, 10)
)

In [18]:
model.load_state_dict(torch.load('/model/model_10_300.pth'))

model.eval()

correct = 0
total = 0

for images, labels in test_data:
    
    images  = images.view(-1, 28 * 28)
    outputs = model(images)
    
    _, predicted = torch.max(outputs.data, 1)
    total += 1
    correct += (predicted == labels).sum()
    
print('Accuracy of test images: %f %%' % (100 * float(correct) / total))

Accuracy of test images: 97.470000 %
